In [2]:
import mysql.connector
import ollama
import pymysql

In [3]:
def get_db_schema(cursor, db_name):
    """Retrieve table schema from MySQL database."""
    cursor.execute(f"SHOW TABLES FROM {db_name}")
    tables = [table[0] for table in cursor.fetchall()]
    schema = {}

    for table in tables:
        cursor.execute(f"DESCRIBE {db_name}.{table}")
        schema[table] = [column[0] for column in cursor.fetchall()]  # Get column names
    
    return schema

def generate_sql_query(user_input, schema, model="mistral"):
    """Use Ollama to generate an SQL query based on user input and schema."""
    prompt = f"""
    You are an expert SQL Assistant. Generate a MySQL query based on the user's request and the given schema.
    
    User Request:
    {user_input}

    Database Schema:
    {schema}

    Ensure the SQL query is optimized, syntactically correct, and formatted properly for MySQL based on the available column names and column data.

    STRICTLY RETURN ONLY THE SQL QUERY FOR EXECUTION AND DON'T HALLUCINATE IF YOU DO NOT GET RESPONSE FROM TABLE !
    """
    
    response = ollama.chat(model=model, messages=[{"role": "user", "content": prompt}])
    return response["message"]["content"].strip()

def execute_sql_query(cursor, query):
    """Execute the generated SQL query and return the results."""
    try:
        cursor.execute(query)
        return cursor.fetchall()
    except mysql.connector.Error as err:
        return f"SQL Execution Error: {err}"

def analyze_results(user_input, sql_query, results, model="mistral"):
    """Use Ollama to analyze SQL query results and provide insights in natural language."""
    prompt = f"""
    You are a data analyst. Analyze the SQL query results and provide a natural language summary.

    User Request:
    {user_input}

    SQL Query:
    {sql_query}

    Query Results:
    {results}

    Explain you analysis in one liner.
    """
    # Provide key insights, patterns, and observations in a concise and structured format in brief.
    response = ollama.chat(model=model, messages=[{"role": "user", "content": prompt}])
    return response["message"]["content"].strip()

def main():
    # Connect to MySQL Database
    db_connection = pymysql.connect(
        host="localhost",
        port=3306,
        user="root",
        password="Ajay@1993",
        database="loandb"
    )
    cursor = db_connection.cursor()

    schema = get_db_schema(cursor, "loandb")
    # results = execute_sql_query(cursor, sql_query)
    
    print('Schema', schema)
    
    user_input = input("Enter your query in natural language as per the Schema: ")

    sql_query = generate_sql_query(user_input, schema, model="mistral")  # Use "mistral" if preferred
    
    sql_query = sql_query.split("```sql")
    sql_query= sql_query[1].split("```")
    sql_query= sql_query[0]

    print("******************************************************************************************************")
    print("\nGenerated SQL Query:\n", sql_query)
    print("******************************************************************************************************")
    
    results = execute_sql_query(cursor, sql_query)
    # print("\nQuery Results:\n", results)

    analysis = analyze_results(user_input, sql_query, results, model="mistral")  # Use "mistral" if preferred
    print("\nAnalysis:\n", analysis)

    cursor.close()
    db_connection.close()

if __name__ == "__main__":
    main()


Schema {'loan': ['Loan_ID', 'Loan_Amount', 'Credit_Score', 'Income', 'Employment_Type', 'Loan_Term', 'Past_Defaults', 'Loan_Status']}


Enter your query in natural language as per the Schema:  Which Employment type has the highest Credit score ?


******************************************************************************************************

Generated SQL Query:
 
SELECT Employment_Type, AVG(Credit_Score) as AverageCreditScore
FROM loan
GROUP BY Employment_Type
ORDER BY AVG(Credit_Score) DESC
LIMIT 1;

******************************************************************************************************

Analysis:
 Among the loan applicants, those employed as Salaried individuals have the highest average credit score of approximately 699.41.


In [4]:
# How many active loans are present ?

In [5]:
# Which class type has the highest income and highest loan ?